In [1]:
import torch
import torch.nn as nn

from model import EnsemblePredictor
import pickle as pk

import os

from config import ensemble_components_folder_path, didi_hidx_traj_path
from config import num_clusters, ensemble_cluster_embedding
from config import num_timeslots, dT

In [18]:
predictors = [torch.load(f'{ensemble_components_folder_path}ensemble_gru_day_{d:02d}.pytorch').cuda(1) for d in range(1, 15)]

In [21]:
ensemble_predictor = EnsemblePredictor(num_clusters, ensemble_cluster_embedding, 14).cuda(1)

optimizer = torch.optim.SGD(ensemble_predictor.parameters(), lr=1e-2)

In [ ]:
T = num_timeslots

batch_size = 512

m = 11
n = 0
avg_loss = 0.0

for d in range(15, 31):

    filename = os.path.join(didi_hidx_traj_path, f'hidx_traj_2012{m:02d}{d:02d}.pk')
    with open(filename, 'rb') as f:
        data = torch.stack(list(pk.load(f).values())).cuda(1)

    for t in range(0, T - 2 * dT, 3):
        
        data = data[torch.randperm(data.shape[0]).cuda(1)]

        if t >= dT:
            optimizer.zero_grad()
            for j in range(1, 4):
                for i in range(0, data.shape[0], batch_size):
                    xc = data[i: i + batch_size, t - dT: t]
                    xt = torch.zeros_like(xc) + t
                    y = data[i: i + batch_size, t + dT - 1]
                    with torch.no_grad():
                        preds = torch.stack([predictor(xc, xt) for predictor in predictors])
                    ensemble_pred = ensemble_predictor(xc[:, -1], preds) 
                    loss = nn.functional.cross_entropy(ensemble_pred, y)
                    loss.backward()
                    optimizer.step()
                    optimizer.zero_grad()

        for i in range(0, data.shape[0], batch_size):
            xc = data[i: i + batch_size, t: t + dT]
            xt = torch.zeros_like(xc) + t
            yc = data[i: i + batch_size, t + 2 * dT - 1]

            with torch.no_grad():
                preds = torch.stack([predictor(xc, xt) for predictor in predictors])
                loss = nn.functional.cross_entropy(ensemble_predictor(xc[:, -1], preds), yc)
                avg_loss += loss.item() * xc.shape[0]
                n += xc.shape[0]
            
            print(f'Day {d:02d} Time {t:02d}, avg_loss={avg_loss / n:.4f}', end='\r')